In [26]:
import pandas as pd
import numpy as np
import seaborn
import tqdm

In [6]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
merchants = pd.read_csv('data/merchants.csv')
new_merchant_transactions = pd.read_csv('data/new_merchant_transactions.csv')
histoy = pd.read_csv('data/historical_transactions.csv')

In [10]:
train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [3]:
test.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [20]:
new_merchant_transactions[new_merchant_transactions['card_id'] == 'C_ID_130fd0cbdd']['purchase_amount'].sum()

-5.944698430000001

In [21]:
histoy[histoy['card_id'] == 'C_ID_130fd0cbdd']['purchase_amount'].sum()

-49.136513300000004

In [29]:
test['card_id'].values.shape

(123623,)

In [30]:
test['new'] = new_merchant_transactions[new_merchant_transactions['card_id']]['purchase_amount'].sum()

KeyError: "None of [Index(['C_ID_415bb3a509', 'C_ID_415bb3a509', 'C_ID_415bb3a509',\n       'C_ID_415bb3a509', 'C_ID_ef55cf8d4b', 'C_ID_ef55cf8d4b',\n       'C_ID_ef55cf8d4b', 'C_ID_ef55cf8d4b', 'C_ID_ef55cf8d4b',\n       'C_ID_ef55cf8d4b',\n       ...\n       'C_ID_c0dda9d36b', 'C_ID_01d35d1005', 'C_ID_611d4d588f',\n       'C_ID_0509e85404', 'C_ID_d3ad1091dc', 'C_ID_1320dee851',\n       'C_ID_f112aa3381', 'C_ID_bd97b86450', 'C_ID_c0513fd84f',\n       'C_ID_a935410f8e'],\n      dtype='object', length=1963031)] are in the [columns]"

In [55]:
new = new_merchant_transactions.groupby('card_id')['purchase_amount'].agg('sum').reset_index()
new.columns = ['card_id', 'purch']

new.head()

,card_id,purch
0,C_ID_00007093c1,-1.328524
1,C_ID_0001238066,-14.850055
2,C_ID_0001506ef0,-1.447354
3,C_ID_0001793786,-0.229620
4,C_ID_000183fdda,-6.590778


In [56]:
his = histoy.groupby('card_id')['purchase_amount'].agg('sum').reset_index()
his.columns = ['card_id', 'purch']

his.head(5)

,card_id,purch
0,C_ID_00007093c1,-76.845041
1,C_ID_0001238066,-72.447201
2,C_ID_0001506ef0,-34.601879
3,C_ID_0001793786,-36.786013
4,C_ID_000183fdda,-68.837938


In [57]:
test_df = pd.merge(test, new, how='left', on='card_id')
test_df = pd.merge(test_df, his, how='left', on='card_id')

In [58]:
test_df.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,purch_x,purch_y
0,2017-04,C_ID_0ab67a22ab,3,3,1,-1.777156,-40.733733
1,2017-01,C_ID_130fd0cbdd,2,3,0,-5.944698,-49.136513
2,2017-08,C_ID_b709037bc5,5,1,1,0.180138,4.528840
3,2017-12,C_ID_d27d835a9f,2,1,0,-5.743674,-13.690715
4,2015-12,C_ID_2b5e3df5c2,5,1,1,12.064997,25.139384


In [59]:
test_df['target'] = test_df['purch_x'] / test_df['purch_y']
test_df.head(5)



,first_active_month,card_id,feature_1,feature_2,feature_3,purch_x,purch_y,target
0,2017-04,C_ID_0ab67a22ab,3,3,1,-1.777156,-40.733733,0.043629
1,2017-01,C_ID_130fd0cbdd,2,3,0,-5.944698,-49.136513,0.120983
2,2017-08,C_ID_b709037bc5,5,1,1,0.180138,4.528840,0.039776
3,2017-12,C_ID_d27d835a9f,2,1,0,-5.743674,-13.690715,0.419531
4,2015-12,C_ID_2b5e3df5c2,5,1,1,12.064997,25.139384,0.479924


In [60]:
test_df['target'] = test_df['target'].fillna(test_df['target'].mean())

In [61]:
pd.DataFrame({'card_id': test_df['card_id'], 'target': test_df['target']}).to_csv('lol_sub.csv', index=False)

In [54]:
test_df

,first_active_month,card_id,feature_1,feature_2,feature_3,purch_x,purch_y,target
0,2017-04,C_ID_0ab67a22ab,3,3,1,-0.592385,-0.599025,0.988915
1,2017-01,C_ID_130fd0cbdd,2,3,0,-0.660522,-0.629955,1.048522
2,2017-08,C_ID_b709037bc5,5,1,1,0.090069,0.348372,0.258542
3,2017-12,C_ID_d27d835a9f,2,1,0,-0.574367,-0.526566,1.090780
4,2015-12,C_ID_2b5e3df5c2,5,1,1,2.010833,0.228540,8.798609
5,2017-07,C_ID_5814b4f13c,5,1,1,-0.175900,-0.052060,3.378782
6,2017-10,C_ID_a1b3c75277,1,2,0,-0.479054,-0.387725,1.235549
7,2017-02,C_ID_f7cada36d3,3,1,1,NaN,-0.689928,1.079419
8,2017-07,C_ID_9d2bc8dfc4,4,3,0,-0.064704,-0.480564,0.134642
9,2016-03,C_ID_6d8dba8475,5,1,1,-0.669384,-0.612941,1.092086
